In [9]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use("seaborn-notebook")

In [10]:
df = pd.read_csv("data/clean_compras.csv")
df.dtypes

NUM BEN           int64
BENEFICIARIO     object
RFC              object
FACTURA          object
CONTRATO         object
CONCEPTO         object
TIPO             object
# OP            float64
FECHA            object
IMPORTE         float64
SHEET            object
dtype: object

## Empezamos con una expansión de datos que tenemos, fecha por ejemplo

In [11]:
df['FECHA'] = pd.to_datetime(df['FECHA'])

In [13]:
df['DIA'] = df['FECHA'].dt.day
df['MES'] = df['FECHA'].dt.month
df['YEAR'] = df['FECHA'].dt.year

## Proyectamos unas variables contra otras

In [21]:
beneficiario_sales = df["BENEFICIARIO"].value_counts().to_dict()
df['BENEFICIARIO_SALES'] = df["BENEFICIARIO"].apply(lambda x : beneficiario_sales[x])
sheet_mean_compras = df.groupby("SHEET")["IMPORTE"].mean()
sheet_mean_compras

SHEET
COMUNICACION              109705.343662
CONTRATISTAS_Y_FDO_FED    474566.765787
GASTOS_VARIOS             881659.731797
SERV._PERS.                59623.405217
SERV_PPROF                229592.166667
Name: IMPORTE, dtype: float64

In [24]:
sheet_mean_compras = sheet_mean_compras.to_dict()
df['IMPORTE_GASTO_PROMEDIO'] = df["SHEET"].apply(lambda x : sheet_mean_compras[x])
df.head()

,NUM BEN,BENEFICIARIO,RFC,FACTURA,CONTRATO,CONCEPTO,TIPO,# OP,FECHA,IMPORTE,SHEET,DIA,MES,YEAR,BENEFICIARIO_SALES,IMPORTE_GASTO_PROMEDIO
0,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS AF8458-158008-3972,OT. 20204373,REEMPLAZO E INSTALACION DE AMORTIGUADORES (DEL...,TR,1019110.0,2020-12-11,14911.80,GASTOS_VARIOS,11,12,2020,13,881659.731797
1,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6916-158008-4041,OT. 20204425,REEMPLAZO E INSTALACION DE OCHO BIRLOS Y NUEVE...,TR,1019110.0,2020-12-11,2528.80,GASTOS_VARIOS,11,12,2020,13,881659.731797
2,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6917-158008-3975,OT. 20204376,REEMPLAZO DE DOS RINES (17). PATRIMONIO 80383...,TR,1019110.0,2020-12-11,8500.02,GASTOS_VARIOS,11,12,2020,13,881659.731797
3,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7004-158008-3971,OT. 20204374,REEMPLAZO DE DOS RINES (14). PATRIMONIO 8007...,TR,1019110.0,2020-12-11,2400.01,GASTOS_VARIOS,11,12,2020,13,881659.731797
4,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7137-158008-4554,OT. 20205130,REEMPLAZO E INSTALACION DE RIN 14. PATRIMONIO ...,TR,299001496.0,2020-12-23,1537.00,GASTOS_VARIOS,23,12,2020,13,881659.731797


## Traernos información externa, enriquecer los datos usando una tool de páginas linkedin

In [25]:
pd.DataFrame(df["BENEFICIARIO"].unique()).to_csv('data/unique_beneficiario.csv', index = None) 

In [32]:
linkedin_mty = pd.read_csv("data/linkedin_mty.csv")
linkedin_mty.head()

,url,description,title,query,timestamp,error
0,https://www.linkedin.com/in/rosa-lorena-valdez...,Ve el perfil de Rosa Lorena Valdez Miranda en ...,Rosa Lorena Valdez Miranda - Analista de Deuda...,Rosa Lorena Valdez Miranda,2021-03-08T13:06:25.587Z,NaN
1,https://www.linkedin.com/in/roque-ya%C3%B1ez-r...,"Ve el perfil de Roque Yañez Ramos en LinkedIn,...",Roque Yañez Ramos - Síndico Primero - Gobierno...,Roque Yañez Ramos,2021-03-08T13:06:52.599Z,NaN
2,https://www.linkedin.com/in/perla-cordova-vill...,Ve el perfil de Perla Cordova Villegas en Link...,Perla Cordova Villegas - Profesor de Tiempo Co...,Laura Perla Cordova Rodriguez,2021-03-08T13:07:18.940Z,NaN
3,https://www.linkedin.com/in/lauren-coronado-b3...,"View Lauren Coronado’s profile on LinkedIn, th...",Lauren Coronado - Substitute Teacher - IRVING ...,Rosa Ofelia Coronado Flores,2021-03-08T13:07:45.390Z,NaN
4,https://www.linkedin.com/in/alvaro-palomo,Ve el perfil de Alvaro Palomo Osuna en LinkedI...,Alvaro Palomo Osuna - Jefe de proyecto .NET - ...,Alvaro Flores Palomo,2021-03-08T13:08:11.744Z,NaN


In [34]:
has_site = {}
for i, ele in linkedin_mty.fillna(True).iterrows():
    has_site[ele['query']] = True if ele['error'] == True else False

In [35]:
has_site

{'Rosa Lorena Valdez Miranda': True,
 'Roque Yañez Ramos': True,
 'Laura Perla Cordova Rodriguez': True,
 'Rosa Ofelia Coronado Flores': True,
 'Alvaro Flores Palomo': True,
 'Liliana Tijerina Cantu': True,
 'Ruben Basaldua Moyeda': True,
 'Maria Cristina Muñoz Rios': True,
 'Gerardo Guerrero Adame': True,
 'Mayela Maria De Lourdes Quiroga Tamez': True,
 'Humberto Arturo Garza De Hoyos': True,
 'Irma Montero Solis': True,
 'Jorge Alejandro Alvarado Martinez': True,
 'Ana Lilia Coronado Araiza': True,
 'Francisco Salazar Guadiana': True,
 'Maria Thalina Almaraz Gonzalez': True,
 'Diego Armando Arellano Aguilar': True,
 'Jorge Adrian Ayala Cantu': True,
 'Adriana Paola Coronado Ramirez': True,
 'LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.': True,
 'GH MAQUINARIA Y EQUIPO, S.A.DE C.V.': True,
 'PLOMIFERRETERA EL TORNILLO, S.A. DE C.V.': False,
 'EQUIPOS Y ACCESORIOS CANTU, S.A. DE C.V.': True,
 'PRAXAIR MEXICO S. DE R.L. DE C.V.': True,
 'COMERCIALIZADORA HNOS.ALVAREZ FLORES,S.A.DE C.V

In [37]:
df['HAS_LINKEDIN_SITE'] = df['BENEFICIARIO'].apply(lambda x: has_site[x.replace("  ", " ")])

In [38]:
df.head()

,NUM BEN,BENEFICIARIO,RFC,FACTURA,CONTRATO,CONCEPTO,TIPO,# OP,FECHA,IMPORTE,SHEET,DIA,MES,YEAR,BENEFICIARIO_SALES,IMPORTE_GASTO_PROMEDIO,HAS_LINKEDIN_SITE
0,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS AF8458-158008-3972,OT. 20204373,REEMPLAZO E INSTALACION DE AMORTIGUADORES (DEL...,TR,1019110.0,2020-12-11,14911.80,GASTOS_VARIOS,11,12,2020,13,881659.731797,True
1,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6916-158008-4041,OT. 20204425,REEMPLAZO E INSTALACION DE OCHO BIRLOS Y NUEVE...,TR,1019110.0,2020-12-11,2528.80,GASTOS_VARIOS,11,12,2020,13,881659.731797,True
2,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF6917-158008-3975,OT. 20204376,REEMPLAZO DE DOS RINES (17). PATRIMONIO 80383...,TR,1019110.0,2020-12-11,8500.02,GASTOS_VARIOS,11,12,2020,13,881659.731797,True
3,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7004-158008-3971,OT. 20204374,REEMPLAZO DE DOS RINES (14). PATRIMONIO 8007...,TR,1019110.0,2020-12-11,2400.01,GASTOS_VARIOS,11,12,2020,13,881659.731797,True
4,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS PF7137-158008-4554,OT. 20205130,REEMPLAZO E INSTALACION DE RIN 14. PATRIMONIO ...,TR,299001496.0,2020-12-23,1537.00,GASTOS_VARIOS,23,12,2020,13,881659.731797,True


In [39]:
df.HAS_LINKEDIN_SITE.value_counts()

True     1392
False     148
Name: HAS_LINKEDIN_SITE, dtype: int64

In [40]:
df.to_csv("data/compras_df_enriched.csv", index = False)